# Getting Started With Machine Learning Using Amazon Sagemaker

## How it works…

#### The SageMaker Python SDK helps data scientists and ML practitioners work on ML experiments using a Python library that abstracts the lower-level API operations, which distinguishes it from the Boto3 AWS SDK for Python. The SageMaker Python SDK makes use of abstraction layers and concepts such as models, estimators, and predictors, with fit() and deploy() functions similar to what libraries and frameworks such as Keras and scikit-learn have.

In [ ]:
from sklearn.model_selection import train_test_split

X = df_all_data['management_experience_months']

X = X.values

y = df_all_data['monthly_salary'].values

     

X_train, X_test, y_train, y_test = train_test_split(

    X, y, test_size=0.3, random_state=0)

In [ ]:
df_training_data = pd.DataFrame({

    'monthly_salary': y_train,

    'management_experience_months': X_train

})

#### This step is important as several algorithms, such as the Linear Learner built-in algorithm, expect the first column to contain the target variable data.

In [ ]:
import sagemaker

import boto3

from sagemaker import get_execution_role

role_arn = get_execution_role()

session = sagemaker.Session()

region_name = boto3.Session().region_name



#### The return values of get_execution_role() and sagemaker.Session() will be used in a later step. The get_execution_role() function from the SageMaker Python SDK returns the IAM role associated with the notebook instance. The return value of this function is used as an argument later when we initialize the Estimator object for the training job.

In [ ]:
training_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/training_data.csv"

training_s3_output_location = f"s3://{s3_bucket}/{prefix}/output/"

from sagemaker.inputs import TrainingInput

train = TrainingInput(training_s3_input_location, content_type="text/csv")

#### Prepare the image URI for Linear Learner. The retrieve() function returns the Amazon ECR URI of the Linear Learner built-in algorithm. Take note that the URI changes depending on the region and the experiments that you are running assume that all resources are in a single region. Otherwise, you will encounter issues during your training jobs. To solve these types of issues, simply specify the region name when using and configuring the different tools:

In [ ]:
from sagemaker.image_uris import retrieve

container = retrieve("linear-learner", region_name, "1")

container

#### Initialize the Estimator object. The Estimator class accepts a couple of arguments, including the container URI, SageMaker session object, and the role ARN we have obtained from the previous steps in this recipe. In the following code, we have also specified the arguments instance_count, instance_type, and output_path:

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=training_s3_output_location,
    sagemaker_session=session)

#### When running training jobs, SageMaker launches new instances outside of the Jupyter notebook instance you are using. These instances are dedicated to running the training jobs and are automatically destroyed after the training jobs have been completed. The number of training job instances used depends on the instance_count argument, and the size and type of the instances depend on the instance_type argument. That said, when the fit() function is called in a later step with this current configuration in the Estimator, SageMaker provisions a single ml.m5.xlarge instance to run the Linear Learner built-algorithm and store the results to output_path.

#### Note that this is one of the ways to initialize an Estimator object and configure training jobs. Another option would be to use the LinearLearner class, which abstracts the container image used when running the training job. We have decided to use the Estimator class in this chapter so that we will have a chance to show that a training job requires a (1) container image, (2) a training dataset, (3) hyperparameters, and (4) a few other configuration values to execute. 

In [ ]:
estimator.set_hyperparameters(predictor_type='regressor', mini_batch_size=4)

#### Hyperparameters are parameter-like values that can be tweaked and configured before the training jobs are executed. The configurable hyperparameter values depend on the algorithm used and, in this case, where we are using the Linear Learner built-in algorithm, we are setting the predictor_type and mini_batch_size hyperparameters.

#### Given that we are dealing with a relatively small training dataset (14 records), we have set the mini_batch_size value to 4. For regression problems, the predictor_type argument should be set to 'regressor', and for classification problems, the predictor_type argument is set to either 'binary_classifier' or 'multiclass_classifier' depending on the number of classes. Each built-in algorithm has its own set of valid hyperparameters and values, so be sure to check the official AWS documentation. As you get more familiar with the built-in algorithms and how they work internally, you will have a better feel for how to modify and tweak these hyperparameters.

#### Execute the training job using the fit() function. This runs the training job by provisioning the servers, running the algorithm on those servers, and then terminating the servers after. As this happens, log messages will be displayed in your notebook to update you on the status of the training job:

In [ ]:
estimator.fit({'train': train})
model_data = estimator.model_data

model_data

#### Use the %store magic to store the value of model_data so that we can use this in the Loading a linear learner model with Apache MXNet in Python and Deploying your first model in Python recipes:

In [ ]:
model_uri = estimator.image_uri

%store model_uri

%store X_test

%store y_test

#### Train-test split: The recipe started with the train-test split, which divided the dataset from the CSV file into the training and test datasets. Ideally, we should perform this step twice so that we will have the training and validation datasets for the model training phase. This way, we can evaluate the model using the test dataset, which was not used for training.

#### Saving and uploading the training dataset to S3: The next step after splitting the dataset is to make sure that the data is formatted properly. Here, we ensured that the first column of the CSV file contains the target value for things to work correctly. The training data is uploaded to Amazon S3 as it is the default source where the training data needs to be uploaded before running the SageMaker training jobs. We used the text/csv content type in this recipe, but we also have the option to use other content types, such as application/x-recordio-protobuf for the Linear Learner built-in algorithm. Using this allows us to use the optimized protobuf recordIO format during the training step and also take advantage of Pipe mode, which improves training job start times for larger datasets.

#### Training: Once the training data has been uploaded to S3, the Estimator object is initialized and configured before starting the training job. The Estimator is simply a high-level interface that allows developers to initialize and configure training jobs with different parameters. Once the training job is complete, the model.tar.gz model file is uploaded to the target S3 bucket.


- The hyperparameters, location of the training input and output files, and other arguments and configuration options specified using the SageMaker Python SDK are passed as parameters to the SageMaker API when using the fit() function.

- ML instances are provisioned by SageMaker and the algorithm container image is pulled and started inside the ML instances. The containers have access to the files that SageMaker has downloaded, including the input files and other configuration options (for example, hyperparameters.json).

- Once the training job has been completed, the output files stored in (/opt/ml/model) are automatically uploaded by SageMaker to the target S3 destination.

In [ ]:
plt.rcParams["figure.figsize"] = (8,8)

plt.scatter(

    df_all_data.management_experience_months,  

    df_all_data.monthly_salary

)

r_line = regression_line_df

plt.plot(r_line['management_experience_months'],  

         r_line['monthly_salary'],

         color='red',

         linewidth=3)

plt.xlabel('Management Experience (Months)', fontsize=18)

plt.ylabel('Monthly Salary (USD)', fontsize=16)

plt.xlim(0, 120)

plt.ylim(0, 2400)

In [ ]:
from sklearn import metrics

import numpy as np

df_test_data = pd.DataFrame({

    'monthly_salary': y_test,

    'management_experience_months': X_test

})

df_test_data

In [ ]:
np.sqrt(metrics.mean_squared_error(df_test_data['monthly_salary'], test_pred))

metrics.mean_squared_error(df_test_data['monthly_salary'], test_pred)

metrics.mean_absolute_error(df_test_data['monthly_salary'], test_pred)

#### When dealing with significantly larger datasets, we can use SageMaker Processing so that we can use dedicated ML instances to run the script that computes for the model evaluation metric values. This is just one of the applications and possible use cases of SageMaker Processing as it can also be used in data preparation and feature engineering.

#### In the previous recipe, we performed the model evaluation step. In this recipe, we will deploy the Linear Learner model to an inference endpoint using the SageMaker Python SDK. What's an inference endpoint? An inference endpoint is a web application endpoint that (1) accepts a set of values as input (for example, x value/s), (2) loads the trained model, (3) uses the trained model to predict a value using the input, and finally, (4) returns the predicted value in the preferred format.

#### After we have deployed the model, we will test the inference endpoint with a few test predictions using sample management_experience_months values. We should get the corresponding predicted monthly_salary values within a second or less!

In [ ]:
import sagemaker

from sagemaker import get_execution_role

     

role = get_execution_role()

session = sagemaker.Session()